In [2]:
import numpy as np
from hurst import compute_Hc
from osgeo import gdal, gdalconst


In [7]:
npp_files = ['./NPP_test/NPP_2001.tif', './NPP_test/NPP_2002.tif', './NPP_test/NPP_2003.tif', './NPP_test/NPP_2004.tif', './NPP_test/NPP_2005.tif', './NPP_test/NPP_2006.tif', './NPP_test/NPP_2007.tif', './NPP_test/NPP_2008.tif', './NPP_test/NPP_2009.tif', './NPP_test/NPP_2010.tif', './NPP_test/NPP_2011.tif', './NPP_test/NPP_2012.tif', './NPP_test/NPP_2013.tif', './NPP_test/NPP_2014.tif', './NPP_test/NPP_2015.tif', './NPP_test/NPP_2016.tif', './NPP_test/NPP_2017.tif', './NPP_test/NPP_2018.tif', './NPP_test/NPP_2019.tif', './NPP_test/NPP_2020.tif', './NPP_test/NPP_2021.tif', './NPP_test/NPP_2022.tif']
output_file = 'hurst_index.tif'


In [8]:
npp_dataset = gdal.Open(npp_files[0], gdal.GA_ReadOnly)
rows, cols = npp_dataset.RasterYSize, npp_dataset.RasterXSize


In [9]:
driver = gdal.GetDriverByName('GTiff')
hurst_dataset = driver.Create(output_file, cols, rows, 1, gdal.GDT_Float32)
hurst_band = hurst_dataset.GetRasterBand(1)
hurst_dataset.SetGeoTransform(npp_dataset.GetGeoTransform())
hurst_dataset.SetProjection(npp_dataset.GetProjection())


0

In [10]:
for i in range(rows):
    for j in range(cols):
        pixel_values = []

        # 从每年的 NPP 文件中读取像素值
        for npp_file in npp_files:
            npp_dataset = gdal.Open(npp_file, gdal.GA_ReadOnly)
            pixel_value = npp_dataset.ReadAsArray(j, i, 1, 1)[0, 0]
            pixel_values.append(pixel_value)

        # 使用 hurst 库计算 Hurst 指数
        if len(pixel_values) > 1:
            H, c, data = compute_Hc(pixel_values)
            hurst_band.WriteArray(np.array(H), j, i)
        else:
            hurst_band.WriteArray(np.array(np.nan), j, i)


ValueError: Series length must be greater or equal to 100

根据错误信息，出现了一个 ValueError，指出系列的长度必须大于等于 100 才能计算 Hurst 指数。这意味着你的时间序列数据 pixel_values 的长度不足 100，无法进行 Hurst 指数的计算。

如果你想要计算 Hurst 指数，你需要有一个足够长的时间序列数据，通常需要包括至少 100 个数据点。如果你的时间序列数据不足 100 个数据点，你可能需要考虑采集更多的数据以满足计算要求。

另外，请确保你的 pixel_values 列表包含正确的时间序列数据，并且每个元素是有效的数值数据。如果 pixel_values 不包含足够的数据点，你需要收集更多的数据来进行计算。

In [ ]:
hurst_band.FlushCache()
hurst_dataset = None
